In [0]:
git clone https://github.com/Ankush96/grad-cam.tensorflow/

SyntaxError: ignored

In [0]:
from vgg import vgg16
import tensorflow as tf
import numpy as np
from skimage import io
from skimage.transform import resize
from matplotlib import pyplot as plt
from imagenet_classes import class_names
from scipy.misc import imread, imresize

flags = tf.app.flags
flags.DEFINE_string("input", "laska.png", "Path to input image ['laska.png']")
flags.DEFINE_string("output", "laska_save.png", "Path to input image ['laska_save.png']")
flags.DEFINE_string("layer_name", "pool5", "Layer till which to backpropagate ['pool5']")

FLAGS = flags.FLAGS


def load_image(img_path):
print("Loading image")
img = imread(img_path, mode='RGB')
img = imresize(img, (224, 224))
# Converting shape from [224,224,3] tp [1,224,224,3]
x = np.expand_dims(img, axis=0)
# Converting RGB to BGR for VGG
x = x[:,:,:,::-1]
return x, img


def grad_cam(x, vgg, sess, predicted_class, layer_name, nb_classes):
print("Setting gradients to 1 for target class and rest to 0")
# Conv layer tensor [?,7,7,512]
conv_layer = vgg.layers[layer_name]
# [1000]-D tensor with target class index set to 1 and rest as 0
one_hot = tf.sparse_to_dense(predicted_class, [nb_classes], 1.0)
signal = tf.mul(vgg.layers['fc3'], one_hot)
loss = tf.reduce_mean(signal)

grads = tf.gradients(loss, conv_layer)[0]
# Normalizing the gradients
norm_grads = tf.div(grads, tf.sqrt(tf.reduce_mean(tf.square(grads))) + tf.constant(1e-5))

output, grads_val = sess.run([conv_layer, norm_grads], feed_dict={vgg.imgs: x})
output = output[0]           # [7,7,512]
grads_val = grads_val[0]	 # [7,7,512]

weights = np.mean(grads_val, axis = (0, 1)) 			# [512]
cam = np.ones(output.shape[0 : 2], dtype = np.float32)	# [7,7]

# Taking a weighted average
for i, w in enumerate(weights):
    cam += w * output[:, :, i]

# Passing through ReLU
cam = np.maximum(cam, 0)
cam = cam / np.max(cam)
cam = resize(cam, (224,224))

# Converting grayscale to 3-D
cam3 = np.expand_dims(cam, axis=2)
cam3 = np.tile(cam3,[1,1,3])

return cam3


def main(_):
x, img = load_image(FLAGS.input)

sess = tf.Session()

print("\nLoading Vgg")
imgs = tf.placeholder(tf.float32, [None, 224, 224, 3])
vgg = vgg16(imgs, 'vgg16_weights.npz', sess)

print("\nFeedforwarding")
prob = sess.run(vgg.probs, feed_dict={vgg.imgs: x})[0]
preds = (np.argsort(prob)[::-1])[0:5]
print('\nTop 5 classes are')
for p in preds:
    print(class_names[p], prob[p])

# Target class
predicted_class = preds[0]
# Target layer for visualization
layer_name = FLAGS.layer_name
# Number of output classes of model being used
nb_classes = 1000

cam3 = grad_cam(x, vgg, sess, predicted_class, layer_name, nb_classes)

img = img.astype(float)
img /= img.max()

# Superimposing the visualization with the image.
new_img = img+3*cam3
new_img /= new_img.max()

# Display and save
io.imshow(new_img)
plt.show()
io.imsave(FLAGS.output, new_img)

if __name__ == '__main__':
tf.app.run()


ImportError: ignored

In [0]:
%run main.py --input laska.png --output laska_save.png --layer_name pool5

ImportError: ignored

In [0]:
from keras.applications.vgg16 import (
    VGG16, preprocess_input, decode_predictions)
from keras.preprocessing import image
from keras.layers.core import Lambda
from keras.models import Sequential
from tensorflow.python.framework import ops
import keras.backend as K
import tensorflow as tf
import numpy as np
import keras
import sys
import cv2

def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, K.one_hot([category_index], nb_classes))

def target_category_loss_output_shape(input_shape):
    return input_shape

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def load_image(path):
    #img_path = 
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

def register_gradient():
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                tf.cast(op.inputs[0] > 0., dtype)

def compile_saliency_function(model, activation_layer='block5_conv3'):
    input_img = model.input
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict[activation_layer].output
    max_output = K.max(layer_output, axis=3)
    saliency = K.gradients(K.sum(max_output), input_img)[0]
    return K.function([input_img, K.learning_phase()], [saliency])

def modify_backprop(model, name):
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': name}):

        # get layers that have an activation
        layer_dict = [layer for layer in model.layers[1:]
                      if hasattr(layer, 'activation')]

        # replace relu activation
        for layer in layer_dict:
            if layer.activation == keras.activations.relu:
                layer.activation = tf.nn.relu

        # re-instanciate a new model
        new_model = VGG16(weights='imagenet')
    return new_model

def deprocess_image(x):
    '''
    Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    '''
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def grad_cam(input_model, image, category_index, layer_name):

    model = tf.keras.Sequential()
    model.add(input_model)

    nb_classes = 1000
    #x = tf.loadLayersModel(model)
    target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
    #target_layer = target_category_loss(layer_name, category_index, nb_classes)
    model.add(Lambda(target_layer,
                     output_shape = target_category_loss_output_shape))
    #print(output_shape)

    loss = K.sum(model.layers[-1].output)
    conv_output =  [l for l in model.layers[0].layers if l.name is layer_name][0].output
    grads = normalize(K.gradients(loss, conv_output)[0])
    gradient_function = K.function([model.layers[0].input], [conv_output, grads])

    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis = (0, 1))
    cam = np.ones(output.shape[0 : 2], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * output[:, :, i]

    cam = cv2.resize(cam, (224, 224))
    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)

    #Return to BGR [0..255] from the preprocessed image
    image = image[0, :]
    image -= np.min(image)
    image = np.minimum(image, 255)

    cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
    cam = np.float32(cam) + np.float32(image)
    cam = 255 * cam / np.max(cam)
    return np.uint8(cam), heatmap

#preprocessed_input = load_image(sys.argv[1])

preprocessed_input = np.zeros((200, 224, 224, 3))

k = 0
for i in range(0, 3):
  preprocessed_input[k] = dip.im_read(img[i]+".jpg")
  k = k+1

model = VGG16(weights='imagenet')
#model = VGG16_MODEL

#pre_inp = dip.im_read(img[10]+".jpg")

predictions = model.predict(preprocessed_input)

#predictions = model.predict(pre_inp)
#top_1 = decode_predictions(predictions)[0][0]
#print('Predicted class:')
#print('%s (%s) with probability %.2f' % (top_1[1], top_1[0], top_1[2]))

predicted_class = np.argmax(predictions)
#predicted_class = history
cam, heatmap = grad_cam(model, preprocessed_input, predicted_class, "block5_conv3")
cv2.imwrite("gradcam.jpg", cam)

register_gradient()
guided_model = modify_backprop(model, 'GuidedBackProp')
saliency_fn = compile_saliency_function(guided_model)
saliency = saliency_fn([preprocessed_input, 0])
gradcam = saliency[0] * heatmap[..., np.newaxis]
cv2.imwrite("guided_gradcam.jpg", deprocess_image(gradcam))

TypeError: ignored

In [0]:
from tensorflow.contrib.slim.nets import vgg

import tensorflow as tf
import missinglink


input_placeholder = tf.placeholder(tf.float32, [None, 224, 224, 3])

VGG16_MODEL=tf.keras.applications.VGG16(input_shape=img_shape,
                                               include_top=False,
                                               weights='imagenet')
VGG16_MODEL.trainable=True
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(n_classes,activation='softmax')
model = tf.keras.Sequential([
  VGG16_MODEL,
  global_average_layer,
  prediction_layer
])

logits, end_points = vgg.vgg_16(inputs=input_placeholder)
probs = tf.nn.softmax(logits)

session = tf.Session()
# Download pre-trained checkpoint from http://download.tensorflow.org/models/vgg_16_2016_08_28.tar.gz
tf.train.Saver().restore(session, 'vgg_16.ckpt')

OWNER_ID = 'replace me with owner id'
PROJECT_TOKEN = 'replace me with project token'

missinglink_project = missinglink.TensorFlowProject(OWNER_ID, PROJECT_TOKEN)

path = 'http://cmeimg-a.akamaihd.net/640/photos.demandstudios.com' + \
    '/getty/article/103/49/516464087.jpg'

with missinglink_project.create_experiment() as experiment:
    experiment.set_properties(
        input_placeholder=input_placeholder.op.name,
        output_layer=probs.op.name,
        last_conv_layer=end_points['vgg_16/conv5/conv5_3'].op.name
    )

    experiment.generate_grad_cam(path, session)

ModuleNotFoundError: ignored

In [0]:
from scipy import ndimage
from pandas_ods_reader import read_ods
import dippykit as dip

path = "./testing/labels_test.ods"
df = read_ods(path, 1, columns=["image", "label"])
img = df.iloc[:, 0].astype('str')
k = 0
for i in img:
  if '.0' in i:
    img[k] = i[0:len(i)-2]
  #temp = dip.im_read("./testing/color_image/"+img[k]+".jpg")
  #temp = ndimage.median_filter(temp, size=15)
  #dip.im_write(temp, "testing/color_image/"+img[k]+".jpg")
  k = k+1

path2 = "./training/labels_train.ods"
df = read_ods(path2, 1, columns=["image", "label"])
img = df.iloc[:, 0].astype('str')
k = 0
for i in img:
  if '.0' in i:
    img[k] = i[0:len(i)-2]
  #temp = dip.im_read("training/color_image/color_image/"+img[k]+".jpg")
  #temp = ndimage.median_filter(temp, size=15)
  #dip.im_write(temp,"training/"+img[k]+".jpg")
  k = k+1

In [0]:
preprocessed_input = dip.im_read("./testing/color_image/"+img[0]+".jpg")

In [0]:
!pip install dippykit
!pip install pandas_ods_reader

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for dippykit: filename=dippykit-2.0.3-cp36-none-any.whl size=58952 sha256=cb3d9511df8dfcc76151a92f9a17c309f8838fdfc4f6584890cc0ab9e4f74c9f
  Stored in directory: /root/.cache/pip/wheels/23/42/9c/a692a4fe24e12db06f80e6e86dcc79cfe784ad832e9a5e96ff
Successfully built dippykit
     |████████████████████████████████| 133kB 4.0MB/s 
  Created wheel for ezodf: filename=ezodf-0.3.2-py2.py3-none-any.whl size=49003 sha256=cddaee2515f069e7ae0c786d325607c58081a945cce310a9b087185ff32be7d0
  Stored in directory: /root/.cache/pip/wheels/c9/6c/f9/107d39d45441980bf273757eba003ef057c6193c9d7650fac7
Successfully built ezodf


In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np
from pandas_ods_reader import read_ods
import dippykit as dip 
from skimage.transform import rescale, resize, downscale_local_mean

# import tensorflow as tf

path = "training/labels_train.ods"
# Add code to read input and convert output values to labels
df = read_ods(path, 1, columns=["image", "label"])
print(df.shape)
img = df.iloc[:, 0].astype('str')
label = df.iloc[:, 1]
lab_train = np.zeros(200)
k = 0
im_train = []
for i in img:
  if '.0' in i:
    img[k] = i[0:len(i)-2]
  # im_train.append("BuildingDataset/training/color_image/"+img[k]+".jpg")
  im_train.append("training/color_image/"+img[k]+".jpg")
  lab_train[k] = label[k]*4
  k = k+1

path2 = "testing/labels_test.ods"
df = read_ods(path2, 1, columns=["image", "label"])
img = df.iloc[:, 0].astype('str')
label_t = df.iloc[:, 1]
lab_test = np.zeros(50)
k = 0
im_test = []
for i in img:
  if '.0' in i:
    img[k] = i[0:len(i)-2]
  # im_test.append("BuildingDataset/testing/color_image/"+img[k]+".jpg")
  im_test.append("testing/color_image/"+img[k]+".jpg")
  lab_test[k] = label_t[k]*4
  k = k+1

(200, 2)


In [0]:
import tensorflow as tf

img_shape = (224, 224, 3)
n_classes = 5
VGG16_MODEL=tf.keras.applications.VGG16(input_shape=img_shape,
                                               include_top=False,
                                               weights='imagenet')
VGG16_MODEL.trainable=True
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(n_classes,activation='softmax')
model = tf.keras.Sequential([
  VGG16_MODEL,
  global_average_layer,
  prediction_layer
])
learning_rate = 0.0005
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=0.01)
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])
# from skimage.util import random_noise
BATCH_SIZE = 50
AUTOTUNE = tf.data.experimental.AUTOTUNE
def _parse_data(x,y):
  # image = dip.im_read(x)
  # image = dip.adjustments.image_noise(image, 's&p')
  # image = tf.convert_to_tensor(image)
  image = tf.read_file(x)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  # image = tf.keras.layers.GaussianNoise(image)
  # image = random_noise(image, 's&p')
  return image, y
  
def _input_fn(x,y):
  s = len(y)
  ds=tf.data.Dataset.from_tensor_slices((x,y))
  ds=ds.map(_parse_data)
  ds=ds.shuffle(s)
  ds = ds.repeat() 
  ds = ds.batch(BATCH_SIZE)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds
  
train_ds=_input_fn(im_train,lab_train)
validation_ds=_input_fn(im_test,lab_test)
history = model.fit(train_ds,
                    epochs=25, 
                    steps_per_epoch=5,
                    validation_steps=5,
                    validation_data=validation_ds)

Train on 5 steps, validate on 5 steps
Epoch 1/25
5/5 [==============================] - 303s 61s/step - loss: 4.2780 - acc: 0.3120 - val_loss: 1.7815 - val_acc: 0.1200
Epoch 2/25
5/5 [==============================] - 301s 60s/step - loss: 1.5143 - acc: 0.2720 - val_loss: 1.5870 - val_acc: 0.3000
Epoch 3/25
5/5 [==============================] - 300s 60s/step - loss: 1.5492 - acc: 0.3200 - val_loss: 1.6148 - val_acc: 0.2800
Epoch 4/25
5/5 [==============================] - 300s 60s/step - loss: 1.4647 - acc: 0.3440 - val_loss: 1.7233 - val_acc: 0.2400
Epoch 5/25
5/5 [==============================] - 300s 60s/step - loss: 1.4448 - acc: 0.3760 - val_loss: 1.6380 - val_acc: 0.2400
Epoch 6/25
5/5 [==============================] - 300s 60s/step - loss: 1.4656 - acc: 0.3480 - val_loss: 1.6599 - val_acc: 0.2400
Epoch 7/25
5/5 [==============================] - 300s 60s/step - loss: 1.4572 - acc: 0.3680 - val_loss: 1.5996 - val_acc: 0.2400
Epoch 8/25
5/5 [==============================] - 30